In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import datgan

from sklearn.preprocessing import OrdinalEncoder

from datgan import stats_assessment
from datgan import ml_assessment, transform_results

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
df_orig = pd.read_csv('../../data/LPMC/trips_regions.csv', index_col=False)

continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt','dur_driving', 'driving_traffic_percent']

cond_inputs = ["age", "female", "hh_regions"]

synth_files = []

for i in range(5):
    synth_files.append('../../data/synthetic/regions/test/DATGAN_{:02d}.csv'.format(i+1))

    synth_files.append('../../data/synthetic/regions/test/ciDATGAN_{:02d}.csv'.format(i+1))

    synth_files.append('../../data/synthetic/regions/test/ciDATGAN_ALL_{:02d}.csv'.format(i+1))


len_df = len(df_orig)

In [3]:
results_path = './results/'

if not os.path.exists(results_path):
    os.makedirs(results_path)

In [4]:
stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']

# First level

In [5]:
pickle_name = 'stats_first_level.pickle'
aggregation_level = 1

first_lvl_stats = {}

try:
    first_lvl_stats = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickle file, using that')
except:
    print('No previous results found, starting fresh')

Found previous pickle file, using that


In [6]:
for i, f in enumerate(synth_files):

    file_name = f.split('/')[-1].split('.')[0]

    if file_name in first_lvl_stats:
        print("Results for file \033[1m{}\033[0m ({}/{}) already exists!".format(file_name, i+1, len(synth_files)))
    else:
        print("Preparing stats for file \033[1m{}\033[0m ({}/{})".format(file_name, i+1, len(synth_files)))

        first_lvl_stats[file_name] = {}

        df_synth = pd.read_csv(f, index_col=False)
        df_synth = df_synth.sample(len_df).reset_index(drop=True)

        stats = stats_assessment(df_orig, df_synth, continuous_columns, aggregation_level, ignore_cols=cond_inputs)

        first_lvl_stats[file_name] = stats

    pickle.dump(first_lvl_stats, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for file DATGAN_01 (1/20) already exists!
Results for file ciDATGAN_01 (2/20) already exists!
Results for file ciDATGAN_ALL_01 (3/20) already exists!
Preparing stats for file ciDATGAN_ALL2_01 (4/20)
Results for file DATGAN_02 (5/20) already exists!
Results for file ciDATGAN_02 (6/20) already exists!
Results for file ciDATGAN_ALL_02 (7/20) already exists!
Preparing stats for file ciDATGAN_ALL2_02 (8/20)
Results for file DATGAN_03 (9/20) already exists!
Results for file ciDATGAN_03 (10/20) already exists!
Results for file ciDATGAN_ALL_03 (11/20) already exists!
Preparing stats for file ciDATGAN_ALL2_03 (12/20)
Results for file DATGAN_04 (13/20) already exists!
Results for file ciDATGAN_04 (14/20) already exists!
Results for file ciDATGAN_ALL_04 (15/20) already exists!
Preparing stats for file ciDATGAN_ALL2_04 (16/20)
Results for file DATGAN_05 (17/20) already exists!
Results for file ciDATGAN_05 (18/20) already exists!
Results for file ciDATGAN_ALL_05 (19/20) already exists!
Prep

In [7]:
res = {}

for test in ['all', 'cont', 'cat']:

    res[test] = {}

    if test == 'all':
        cols = df_orig.columns
    elif test == 'cont':
        cols = continuous_columns
    elif test == 'cat':
        cols = set(df_orig.columns) - set(continuous_columns)

    cols = set(cols) - set(cond_inputs)

    for s in stats_str:
        res[test][s] = {}

    for m in first_lvl_stats.keys():
        for s in stats_str:
            res[test][s][m] = []

            for c in cols:
                res[test][s][m].append(first_lvl_stats[m][c][s])

In [8]:
avg = {}

for test in ['all', 'cont', 'cat']:

    avg[test] = {}

    for s in stats_str:
        avg[test][s] = {}

        for m in first_lvl_stats.keys():
            avg[test][s][m] = {
                'mean': np.mean(res[test][s][m]),
                'std': np.std(res[test][s][m])
            }

In [9]:
for test in ['all', 'cont', 'cat']:

    if test == 'all':
        str_ = 'on all columns'
    elif test == 'cont':
        str_ = 'on continuous columns'
    elif test == 'cat':
        str_ = 'on categorical columns'

    for s in ['srmse']:#stats:
        print('Ranking {} based on {}:'.format(str_, s.upper()))

        if s in ['r2', 'corr']:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])[::-1]}
        else:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])}

        for i, item in enumerate(sorted_dct):
            print('  {:>2}. {:<15} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
        print()

Ranking on all columns based on SRMSE:
   1. ciDATGAN_01     - 6.06e-02 ± 3.96e-02
   2. ciDATGAN_05     - 6.08e-02 ± 3.90e-02
   3. ciDATGAN_04     - 6.28e-02 ± 4.16e-02
   4. DATGAN_04       - 6.32e-02 ± 4.18e-02
   5. DATGAN_03       - 6.32e-02 ± 4.53e-02
   6. ciDATGAN_02     - 6.34e-02 ± 4.30e-02
   7. DATGAN_01       - 6.36e-02 ± 4.50e-02
   8. DATGAN_05       - 6.47e-02 ± 4.47e-02
   9. ciDATGAN_03     - 6.58e-02 ± 4.11e-02
  10. DATGAN_02       - 6.83e-02 ± 4.86e-02
  11. ciDATGAN_ALL2_05 - 7.48e-02 ± 8.29e-02
  12. ciDATGAN_ALL2_03 - 7.50e-02 ± 8.44e-02
  13. ciDATGAN_ALL2_02 - 7.53e-02 ± 7.97e-02
  14. ciDATGAN_ALL2_04 - 7.61e-02 ± 8.69e-02
  15. ciDATGAN_ALL2_01 - 7.94e-02 ± 8.39e-02
  16. ciDATGAN_ALL_03 - 9.54e-02 ± 9.70e-02
  17. ciDATGAN_ALL_02 - 9.54e-02 ± 9.61e-02
  18. ciDATGAN_ALL_05 - 9.70e-02 ± 9.74e-02
  19. ciDATGAN_ALL_01 - 9.70e-02 ± 9.73e-02
  20. ciDATGAN_ALL_04 - 9.87e-02 ± 9.96e-02

Ranking on continuous columns based on SRMSE:
   1. ciDATGAN_01     - 9.46e

# Second level

In [10]:
pickle_name = 'stats_second_level.pickle'
aggregation_level = 2

second_lvl_stats = {}

try:
    second_lvl_stats = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')

Found previous pickel file, using that


In [11]:
for i, f in enumerate(synth_files):

    file_name = f.split('/')[-1].split('.')[0]

    if file_name in second_lvl_stats:
        print("Results for file \033[1m{}\033[0m ({}/{}) already exists!".format(file_name, i+1, len(synth_files)))
    else:
        print("Preparing stats for file \033[1m{}\033[0m ({}/{})".format(file_name, i+1, len(synth_files)))

        second_lvl_stats[file_name] = {}

        df_synth = pd.read_csv(f, index_col=False)
        df_synth = df_synth.sample(len_df).reset_index(drop=True)

        stats = stats_assessment(df_orig, df_synth, continuous_columns, aggregation_level, ignore_cols=cond_inputs)

        second_lvl_stats[file_name] = stats

    pickle.dump(second_lvl_stats, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for file DATGAN_01 (1/20) already exists!
Results for file ciDATGAN_01 (2/20) already exists!
Results for file ciDATGAN_ALL_01 (3/20) already exists!
Preparing stats for file ciDATGAN_ALL2_01 (4/20)
Results for file DATGAN_02 (5/20) already exists!
Results for file ciDATGAN_02 (6/20) already exists!
Results for file ciDATGAN_ALL_02 (7/20) already exists!
Preparing stats for file ciDATGAN_ALL2_02 (8/20)
Results for file DATGAN_03 (9/20) already exists!
Results for file ciDATGAN_03 (10/20) already exists!
Results for file ciDATGAN_ALL_03 (11/20) already exists!
Preparing stats for file ciDATGAN_ALL2_03 (12/20)
Results for file DATGAN_04 (13/20) already exists!
Results for file ciDATGAN_04 (14/20) already exists!
Results for file ciDATGAN_ALL_04 (15/20) already exists!
Preparing stats for file ciDATGAN_ALL2_04 (16/20)
Results for file DATGAN_05 (17/20) already exists!
Results for file ciDATGAN_05 (18/20) already exists!
Results for file ciDATGAN_ALL_05 (19/20) already exists!
Prep

In [12]:
res = {}

for s in stats_str:
    res[s] = {}

for m in second_lvl_stats.keys():

    for s in stats_str:
        res[s][m] = []

        for c in second_lvl_stats[m].keys():
            res[s][m].append(second_lvl_stats[m][c][s])

In [13]:
avg = {}

for s in stats_str:
    avg[s] = {}

    for m in first_lvl_stats.keys():
        avg[s][m] = {
            'mean': np.mean(res[s][m]),
            'std': np.std(res[s][m])
        }

In [14]:
for s in ['srmse']:#stats:
    print('Ranking based on {} for aggregation level {}:'.format(s.upper(), aggregation_level))

    if s in ['r2', 'corr']:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])[::-1]}
    else:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])}

    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<15} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
    print()

Ranking based on SRMSE for aggregation level 2:
   1. ciDATGAN_05     - 1.53e-01 ± 6.62e-02
   2. ciDATGAN_01     - 1.53e-01 ± 6.45e-02
   3. ciDATGAN_04     - 1.56e-01 ± 6.55e-02
   4. ciDATGAN_02     - 1.58e-01 ± 6.67e-02
   5. ciDATGAN_03     - 1.60e-01 ± 6.52e-02
   6. DATGAN_04       - 1.70e-01 ± 6.54e-02
   7. DATGAN_01       - 1.73e-01 ± 6.97e-02
   8. DATGAN_05       - 1.76e-01 ± 6.57e-02
   9. DATGAN_03       - 1.76e-01 ± 7.15e-02
  10. DATGAN_02       - 1.81e-01 ± 7.28e-02
  11. ciDATGAN_ALL2_02 - 2.08e-01 ± 1.29e-01
  12. ciDATGAN_ALL2_03 - 2.10e-01 ± 1.38e-01
  13. ciDATGAN_ALL2_05 - 2.10e-01 ± 1.35e-01
  14. ciDATGAN_ALL2_04 - 2.10e-01 ± 1.41e-01
  15. ciDATGAN_ALL2_01 - 2.16e-01 ± 1.37e-01
  16. ciDATGAN_ALL_03 - 2.25e-01 ± 1.76e-01
  17. ciDATGAN_ALL_02 - 2.27e-01 ± 1.73e-01
  18. ciDATGAN_ALL_05 - 2.30e-01 ± 1.75e-01
  19. ciDATGAN_ALL_01 - 2.31e-01 ± 1.74e-01
  20. ciDATGAN_ALL_04 - 2.33e-01 ± 1.79e-01



# ML efficacy

In [15]:
def check_low_appearing_vars(df):

    for c in df.columns:
        val = df[c].value_counts()
        if len(val) < 20:
            val = val/len(df)
            if any(val < 0.01) and c != 'choice':
                print('Variable {}: '.format(c))
                for idx, v in zip(val.index, val):
                    if v < 0.01:
                        print('  {} - {:.2f}% ({:d})'.format(idx, 100*v, int(v*len(df))))
                print()

def replace_low_appearing_values(df):

    dct_ = {}
    for i in df['hh_vehicles'].unique():
        if i >= 3:
            dct_[i] = '3+'
        else:
            dct_[i] = str(i)
    df['hh_vehicles'].replace(dct_, inplace=True)

    dct_ = {}
    for i in df['hh_people'].unique():
        if i >= 6:
            dct_[i] = '6+'
        else:
            dct_[i] = str(i)
    df['hh_people'].replace(dct_, inplace=True)

In [16]:
check_low_appearing_vars(df_orig)

Variable hh_vehicles: 
  4 - 0.52% (88)
  5 - 0.12% (21)
  6 - 0.02% (3)
  8 - 0.02% (3)
  7 - 0.01% (2)

Variable hh_people: 
  7 - 0.70% (118)
  8 - 0.18% (31)
  9 - 0.08% (13)
  10 - 0.03% (5)
  11 - 0.02% (3)



In [17]:
replace_low_appearing_values(df_orig)

In [18]:
check_low_appearing_vars(df_orig)

In [19]:
categorical_columns = list(set(df_orig.columns) - set(continuous_columns))

In [20]:
enc = OrdinalEncoder()
df_orig[categorical_columns] = enc.fit_transform(df_orig[categorical_columns])

In [21]:
pickle_name = 'ml_efficacy.pickle'

cv_modelscores = {}

try:
    cv_modelscores = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')

Found previous pickel file, using that


In [22]:
for i, f in enumerate(synth_files):

    file_name = f.split('/')[-1].split('.')[0]

    if file_name in cv_modelscores:
        print("Results for file \033[1m{}\033[0m ({}/{}) already exists!".format(file_name, i+1, len(synth_files)))
    else:
        print("Preparing stats for file \033[1m{}\033[0m ({}/{})".format(file_name, i+1, len(synth_files)))

        cv_modelscores[file_name] = {}

        # Load the synthetic dataset
        df_synth = pd.read_csv(f, index_col=False)
        df_synth = df_synth.sample(len_df).reset_index(drop=True)

        # Replace the values rarely appearing
        replace_low_appearing_values(df_synth)

        # Encode the synthetic dataset
        df_synth[categorical_columns] = enc.transform(df_synth[categorical_columns])

        res = ml_assessment(df_orig, df_synth, continuous_columns, categorical_columns, ignore_cols=cond_inputs)

        cv_modelscores[file_name] = res

    pickle.dump(cv_modelscores, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for file DATGAN_01 (1/20) already exists!
Results for file ciDATGAN_01 (2/20) already exists!
Preparing stats for file ciDATGAN_ALL_01 (3/20)
Preparing stats for file ciDATGAN_ALL2_01 (4/20)
Results for file DATGAN_02 (5/20) already exists!
Results for file ciDATGAN_02 (6/20) already exists!
Preparing stats for file ciDATGAN_ALL_02 (7/20)
Preparing stats for file ciDATGAN_ALL2_02 (8/20)
Results for file DATGAN_03 (9/20) already exists!
Results for file ciDATGAN_03 (10/20) already exists!
Preparing stats for file ciDATGAN_ALL_03 (11/20)
Preparing stats for file ciDATGAN_ALL2_03 (12/20)
Results for file DATGAN_04 (13/20) already exists!
Results for file ciDATGAN_04 (14/20) already exists!
Preparing stats for file ciDATGAN_ALL_04 (15/20)
Preparing stats for file ciDATGAN_ALL2_04 (16/20)
Results for file DATGAN_05 (17/20) already exists!
Results for file ciDATGAN_05 (18/20) already exists!
Preparing stats for file ciDATGAN_ALL_05 (19/20)
Preparing stats for file ciDATGAN_ALL2_05 (2

In [23]:
if 'original' in cv_modelscores:
    print("Results for file \033[1m{}\033[0m already exists!".format('original'))
else:
    print("Preparing stats for file \033[1m{}\033[0m".format('original'))

    res = ml_assessment(df_orig, df_orig, continuous_columns, categorical_columns, ignore_cols=cond_inputs)
    cv_modelscores['original'] = res
    pickle.dump(cv_modelscores, open(results_path + pickle_name, 'wb'))
    print("\033[1mFINISHED!\033[0m")

Results for file original already exists!


In [24]:
cont_sorted, cat_sorted = transform_results(cv_modelscores, continuous_columns, categorical_columns, ignore_cols=cond_inputs)

In [25]:
i=1
print('   | {:<30} | {:<30}'.format('categorical', 'continuous'))
print('-----------------------------------------------------------')
for a, b in zip(cat_sorted, cont_sorted):
    print('{:>2} | {:<30} | {:<30}'.format(i, '{:<12}: {:.3f}'.format(a[0], a[1]), '{:<12}: {:.3f}'.format(b[0], b[1])))
    i+=1

   | categorical                    | continuous                    
-----------------------------------------------------------
 1 | original    : -1.536           | original    : 5.385           
 2 | ciDATGAN_ALL_04: 0.557         | ciDATGAN_ALL_03: 11.584       
 3 | ciDATGAN_ALL_01: 0.568         | ciDATGAN_ALL_04: 11.689       
 4 | ciDATGAN_ALL_02: 0.575         | ciDATGAN_ALL_02: 11.780       
 5 | ciDATGAN_ALL_05: 0.594         | ciDATGAN_ALL_05: 11.801       
 6 | ciDATGAN_ALL_03: 0.605         | ciDATGAN_ALL_01: 11.922       
 7 | ciDATGAN_05 : 0.641            | DATGAN_02   : 12.649          
 8 | ciDATGAN_04 : 0.645            | DATGAN_03   : 12.712          
 9 | ciDATGAN_01 : 0.654            | DATGAN_01   : 12.898          
10 | ciDATGAN_03 : 0.654            | DATGAN_04   : 12.949          
11 | ciDATGAN_02 : 0.664            | DATGAN_05   : 12.971          
12 | ciDATGAN_ALL2_03: 0.771        | ciDATGAN_ALL2_04: 14.463      
13 | ciDATGAN_ALL2_02: 0.786        | ciDAT